In [1]:
import pandas as pd
import numpy as np
from snowflake.sqlalchemy import URL
from datetime import datetime
import warnings
from dateutil.relativedelta import relativedelta

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

/Users/keyurpethad/anaconda3/lib/python3.11/site-packages/snowflake/connector/options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (11.0.0), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(


#### Reading data

In [15]:
application_train = pd.read_csv('data/application_train.csv')
application_test = pd.read_csv('data/application_test.csv')
bureau_balance = pd.read_csv('data/bureau_balance.csv')
bureau = pd.read_csv('data/bureau.csv')
credit_card_balance = pd.read_csv('data/credit_card_balance.csv')
installments_payments = pd.read_csv('data/installments_payments.csv')
POS_CASH_balance = pd.read_csv('data/POS_CASH_balance.csv')
previous_application = pd.read_csv('data/previous_application.csv')

#### Application var creation 

In [16]:
application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming application_train is your DataFrame
# Replace 'YourCategoricalColumn' and 'YourNumericalColumn' with the actual column names

# Display value counts for categorical columns
categorical_columns = application_train.select_dtypes(include=['object']).columns
for col in categorical_columns:
    print(f"Value counts for {col}:")
    print(application_train[col].value_counts(dropna=False))
    print("\n")

# Display separate histograms for numerical columns
numerical_columns = application_train.select_dtypes(include=['int64', 'float64']).columns
for col in numerical_columns:
    print(col)
    plt.figure(figsize=(8, 6))
    application_train[col].hist(bins=20)
    plt.title(f'Histogram for {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.show()


In [ ]:
def application_preproc(df):
    # NaN values for DAYS_EMPLOYED: 365243 -> nan
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    # NaN values for OWN_CAR_AGE: 91 -> nan
    df['DAYS_EMPLOYED'].replace(91, np.nan, inplace= True)
    # Some simple new features (percentages)
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']

    df = df.drop(columns = {'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'ORGANIZATION_TYPE'})

    return df

In [27]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

#### Bureau balance var creation

In [79]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [80]:
bureau_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [81]:
bb, bb_cat = one_hot_encoder(bureau_balance, True)

In [82]:
# Bureau balance: Perform aggregations and merge with bureau.csv
bb_aggregations = dict()
for col in bb_cat:
    bb_aggregations[col] = ['any','mean']
bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)

# Getting tenure related info when loan was open
bb_agg_open = bb[bb['STATUS_C'] == False].groupby('SK_ID_BUREAU').agg({'MONTHS_BALANCE': ['min', 'max', 'size']})
bb_agg = bb_agg.merge(bb_agg_open, how = 'left', on = 'SK_ID_BUREAU')

In [83]:
bb_agg.head()

STATUS_0           STATUS_1      STATUS_2      STATUS_3       \
                  any      mean      any mean      any mean      any mean   
SK_ID_BUREAU                                                                
5001709         False  0.000000    False  0.0    False  0.0    False  0.0   
5001710          True  0.060241    False  0.0    False  0.0    False  0.0   
5001711          True  0.750000    False  0.0    False  0.0    False  0.0   
5001712          True  0.526316    False  0.0    False  0.0    False  0.0   
5001713         False  0.000000    False  0.0    False  0.0    False  0.0   

             STATUS_4      STATUS_5      STATUS_C           STATUS_X  \
                  any mean      any mean      any      mean      any   
SK_ID_BUREAU                                                           
5001709         False  0.0    False  0.0     True  0.886598     True   
5001710         False  0.0    False  0.0     True  0.578313     True   
5001711         False  0.0    False  0.0    False  0.000000     True   
5001712         False  0.0    False  0.0     True  0.473684    False   
5001713         False  0.0    False  0.0    False  0.000000     True   

                       STATUS_nan      MONTHS_BALANCE              
                  mean        any mean            min   max  size  
SK_ID_BUREAU                                                       
5001709       0.113402      False  0.0          -96.0 -86.0  11.0  
5001710       0.361446      False  0.0          -82.0 -48.0  35.0  
5001711       0.250000      False  0.0           -3.0   0.0   4.0  
5001712       0.000000      False  0.0          -18.0  -9.0  10.0  
5001713       1.000000      False  0.0          -21.0   0.0  22.0

In [84]:
bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])

In [85]:
bb_agg

,STATUS_0_ANY,STATUS_0_MEAN,STATUS_1_ANY,STATUS_1_MEAN,STATUS_2_ANY,STATUS_2_MEAN,STATUS_3_ANY,STATUS_3_MEAN,STATUS_4_ANY,STATUS_4_MEAN,STATUS_5_ANY,STATUS_5_MEAN,STATUS_C_ANY,STATUS_C_MEAN,STATUS_X_ANY,STATUS_X_MEAN,STATUS_nan_ANY,STATUS_nan_MEAN,MONTHS_BALANCE_MIN,MONTHS_BALANCE_MAX,MONTHS_BALANCE_SIZE
SK_ID_BUREAU,,,,,,,,,,,,,,,,,,,,,
5001709,False,0.000000,False,0.000000,False,0.0,False,0.0,False,0.0,False,0.0,True,0.886598,True,0.113402,False,0.0,-96.0,-86.0,11.0
5001710,True,0.060241,False,0.000000,False,0.0,False,0.0,False,0.0,False,0.0,True,0.578313,True,0.361446,False,0.0,-82.0,-48.0,35.0
5001711,True,0.750000,False,0.000000,False,0.0,False,0.0,False,0.0,False,0.0,False,0.000000,True,0.250000,False,0.0,-3.0,0.0,4.0
5001712,True,0.526316,False,0.000000,False,0.0,False,0.0,False,0.0,False,0.0,True,0.473684,False,0.000000,False,0.0,-18.0,-9.0,10.0
5001713,False,0.000000,False,0.000000,False,0.0,False,0.0,False,0.0,False,0.0,False,0.000000,True,1.000000,False,0.0,-21.0,0.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6842884,True,0.187500,False,0.000000,False,0.0,False,0.0,False,0.0,False,0.0,True,0.416667,True,0.395833,False,0.0,-47.0,-20.0,28.0
6842885,True,0.500000,False,0.000000,False,0.0,False,0.0,False,0.0,True,0.5,False,0.000000,False,0.000000,False,0.0,-23.0,0.0,24.0
6842886,True,0.242424,False,0.000000,False,0.0,False,0.0,False,0.0,False,0.0,True,0.757576,False,0.000000,False,0.0,-32.0,-25.0,8.0


In [86]:
bureau = bureau.merge(bb_agg.reset_index(), how = 'left', on = 'SK_ID_BUREAU')

In [89]:
bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
del bb, bb_agg, bb_agg_open

#### Bureau var creation

In [94]:
bureau['CREDIT_TYPE'] = np.where(bureau['CREDIT_TYPE'].isin(['Consumer credit']), 'consumer_credit',
                            np.where(bureau['CREDIT_TYPE'].isin(['Credit card']), 'credit_card',
                                np.where(bureau['CREDIT_TYPE'].isin(['Car loan']), 'car_loan',
                                    np.where(bureau['CREDIT_TYPE'].isin(['Mortgage', 'Real estate loan']), 'home_loan',
                                        np.where(bureau['CREDIT_TYPE'].isin(['Microloan']), 'Microloan', 'Other')))))

In [95]:
bureau['CREDIT_TYPE'].value_counts()

CREDIT_TYPE
consumer_credit    1251615
credit_card         402195
car_loan             27690
home_loan            18418
Microloan            12413
Other                 4097
Name: count, dtype: int64

In [97]:
bureau.head()

,SK_ID_CURR,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,STATUS_0_ANY,STATUS_0_MEAN,STATUS_1_ANY,STATUS_1_MEAN,STATUS_2_ANY,STATUS_2_MEAN,STATUS_3_ANY,STATUS_3_MEAN,STATUS_4_ANY,STATUS_4_MEAN,STATUS_5_ANY,STATUS_5_MEAN,STATUS_C_ANY,STATUS_C_MEAN,STATUS_X_ANY,STATUS_X_MEAN,STATUS_nan_ANY,STATUS_nan_MEAN,MONTHS_BALANCE_MIN,MONTHS_BALANCE_MAX,MONTHS_BALANCE_SIZE
0,215354,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,consumer_credit,-131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,215354,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,credit_card,-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,215354,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,consumer_credit,-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,215354,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,credit_card,-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,215354,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,consumer_credit,-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
aggregation = {'DAYS_CREDIT': ['max', 'min'],
'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
'DAYS_CREDIT_UPDATE': ['mean', 'min'],
'CREDIT_DAY_OVERDUE': ['max', 'mean'],
'AMT_CREDIT_MAX_OVERDUE': ['mean', 'max'],
'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
'AMT_CREDIT_SUM_OVERDUE': ['mean', 'max'],
'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
'AMT_ANNUITY': ['max', 'mean'],
'CNT_CREDIT_PROLONG': ['sum'],
'CNT_CREDIT_PROLONG': ['sum', 'any'],
'MONTHS_BALANCE_MIN': ['min'],
'MONTHS_BALANCE_MAX': ['max'],
'MONTHS_BALANCE_SIZE': ['mean'],
'STATUS_0_ANY': ['any', 'mean'],
'STATUS_0_MEAN': ['mean'],
'STATUS_1_ANY': ['any', 'mean'],
'STATUS_1_MEAN': ['mean'],
'STATUS_2_ANY': ['any', 'mean'],
'STATUS_2_MEAN': ['mean'],
'STATUS_3_ANY': ['any', 'mean'],
'STATUS_3_MEAN': ['mean'],
'STATUS_4_ANY': ['any', 'mean'],
'STATUS_4_MEAN': ['mean'],
'STATUS_C_ANY': ['any', 'mean'],
'STATUS_C_MEAN': ['mean'],
'STATUS_X_ANY': ['any', 'mean'],
'STATUS_X_MEAN': ['mean'],
'STATUS_nan_ANY': ['any', 'mean'],
'STATUS_nan_MEAN': ['mean'],
}

In [105]:
bureau_agg = bureau.groupby('SK_ID_CURR').agg(aggregation)
bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])

active = bureau[bureau['CREDIT_ACTIVE'] == 'Active']
active_agg = active.groupby('SK_ID_CURR').agg(aggregation)
active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')

for loan_type in bureau['CREDIT_TYPE'].value_counts().index[:-1]:
    credit_ty = bureau[bureau['CREDIT_TYPE'] == loan_type]
    credit_ty_agg = credit_ty.groupby('SK_ID_CURR').agg(aggregation)
    credit_ty_agg.columns = pd.Index([loan_type + "_" + e[0] + "_" + e[1].upper() for e in credit_ty_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(credit_ty_agg, how='left', on='SK_ID_CURR')

curr1 = bureau[bureau['CREDIT_CURRENCY'] == 'currency 1']
curr1_agg = curr1.groupby('SK_ID_CURR').agg(aggregation)
curr1_agg.columns = pd.Index(['curr1_' + e[0] + "_" + e[1].upper() for e in curr1_agg.columns.tolist()])
bureau_agg = bureau_agg.join(curr1_agg, how='left', on='SK_ID_CURR')

currn1 = bureau[bureau['CREDIT_CURRENCY'] != 'currency 1']
currn1_agg = currn1.groupby('SK_ID_CURR').agg(aggregation)
currn1_agg.columns = pd.Index(['currn1_' + e[0] + "_" + e[1].upper() for e in currn1_agg.columns.tolist()])
bureau_agg = bureau_agg.join(currn1_agg, how='left', on='SK_ID_CURR')

del active, active_agg, credit_ty, credit_ty_agg, curr1, curr1_agg, currn1, currn1_agg

In [106]:
bureau_agg.head()

,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_ENDDATE_MIN,BURO_DAYS_CREDIT_ENDDATE_MAX,BURO_DAYS_CREDIT_ENDDATE_MEAN,BURO_DAYS_CREDIT_UPDATE_MEAN,BURO_DAYS_CREDIT_UPDATE_MIN,BURO_CREDIT_DAY_OVERDUE_MAX,BURO_CREDIT_DAY_OVERDUE_MEAN,BURO_AMT_CREDIT_MAX_OVERDUE_MEAN,BURO_AMT_CREDIT_MAX_OVERDUE_MAX,BURO_AMT_CREDIT_SUM_MAX,BURO_AMT_CREDIT_SUM_MEAN,BURO_AMT_CREDIT_SUM_SUM,BURO_AMT_CREDIT_SUM_DEBT_MAX,BURO_AMT_CREDIT_SUM_DEBT_MEAN,BURO_AMT_CREDIT_SUM_DEBT_SUM,BURO_AMT_CREDIT_SUM_OVERDUE_MEAN,BURO_AMT_CREDIT_SUM_OVERDUE_MAX,BURO_AMT_CREDIT_SUM_LIMIT_MEAN,BURO_AMT_CREDIT_SUM_LIMIT_SUM,BURO_AMT_ANNUITY_MAX,BURO_AMT_ANNUITY_MEAN,BURO_CNT_CREDIT_PROLONG_SUM,BURO_CNT_CREDIT_PROLONG_ANY,BURO_MONTHS_BALANCE_MIN_MIN,BURO_MONTHS_BALANCE_MAX_MAX,BURO_MONTHS_BALANCE_SIZE_MEAN,BURO_STATUS_0_ANY_ANY,BURO_STATUS_0_ANY_MEAN,BURO_STATUS_0_MEAN_MEAN,BURO_STATUS_1_ANY_ANY,BURO_STATUS_1_ANY_MEAN,BURO_STATUS_1_MEAN_MEAN,BURO_STATUS_2_ANY_ANY,BURO_STATUS_2_ANY_MEAN,BURO_STATUS_2_MEAN_MEAN,BURO_STATUS_3_ANY_ANY,BURO_STATUS_3_ANY_MEAN,BURO_STATUS_3_MEAN_MEAN,BURO_STATUS_4_ANY_ANY,BURO_STATUS_4_ANY_MEAN,BURO_STATUS_4_MEAN_MEAN,BURO_STATUS_C_ANY_ANY,BURO_STATUS_C_ANY_MEAN,BURO_STATUS_C_MEAN_MEAN,BURO_STATUS_X_ANY_ANY,BURO_STATUS_X_ANY_MEAN,BURO_STATUS_X_MEAN_MEAN,BURO_STATUS_nan_ANY_ANY,BURO_STATUS_nan_ANY_MEAN,BURO_STATUS_nan_MEAN_MEAN,ACTIVE_DAYS_CREDIT_MAX,ACTIVE_DAYS_CREDIT_MIN,ACTIVE_DAYS_CREDIT_ENDDATE_MIN,ACTIVE_DAYS_CREDIT_ENDDATE_MAX,ACTIVE_DAYS_CREDIT_ENDDATE_MEAN,ACTIVE_DAYS_CREDIT_UPDATE_MEAN,ACTIVE_DAYS_CREDIT_UPDATE_MIN,ACTIVE_CREDIT_DAY_OVERDUE_MAX,ACTIVE_CREDIT_DAY_OVERDUE_MEAN,ACTIVE_AMT_CREDIT_MAX_OVERDUE_MEAN,ACTIVE_AMT_CREDIT_MAX_OVERDUE_MAX,ACTIVE_AMT_CREDIT_SUM_MAX,ACTIVE_AMT_CREDIT_SUM_MEAN,ACTIVE_AMT_CREDIT_SUM_SUM,ACTIVE_AMT_CREDIT_SUM_DEBT_MAX,ACTIVE_AMT_CREDIT_SUM_DEBT_MEAN,ACTIVE_AMT_CREDIT_SUM_DEBT_SUM,ACTIVE_AMT_CREDIT_SUM_OVERDUE_MEAN,ACTIVE_AMT_CREDIT_SUM_OVERDUE_MAX,ACTIVE_AMT_CREDIT_SUM_LIMIT_MEAN,ACTIVE_AMT_CREDIT_SUM_LIMIT_SUM,ACTIVE_AMT_ANNUITY_MAX,ACTIVE_AMT_ANNUITY_MEAN,ACTIVE_CNT_CREDIT_PROLONG_SUM,ACTIVE_CNT_CREDIT_PROLONG_ANY,ACTIVE_MONTHS_BALANCE_MIN_MIN,ACTIVE_MONTHS_BALANCE_MAX_MAX,ACTIVE_MONTHS_BALANCE_SIZE_MEAN,ACTIVE_STATUS_0_ANY_ANY,ACTIVE_STATUS_0_ANY_MEAN,ACTIVE_STATUS_0_MEAN_MEAN,ACTIVE_STATUS_1_ANY_ANY,ACTIVE_STATUS_1_ANY_MEAN,ACTIVE_STATUS_1_MEAN_MEAN,ACTIVE_STATUS_2_ANY_ANY,ACTIVE_STATUS_2_ANY_MEAN,ACTIVE_STATUS_2_MEAN_MEAN,ACTIVE_STATUS_3_ANY_ANY,ACTIVE_STATUS_3_ANY_MEAN,ACTIVE_STATUS_3_MEAN_MEAN,ACTIVE_STATUS_4_ANY_ANY,ACTIVE_STATUS_4_ANY_MEAN,ACTIVE_STATUS_4_MEAN_MEAN,ACTIVE_STATUS_C_ANY_ANY,ACTIVE_STATUS_C_ANY_MEAN,ACTIVE_STATUS_C_MEAN_MEAN,ACTIVE_STATUS_X_ANY_ANY,ACTIVE_STATUS_X_ANY_MEAN,ACTIVE_STATUS_X_MEAN_MEAN,ACTIVE_STATUS_nan_ANY_ANY,ACTIVE_STATUS_nan_ANY_MEAN,ACTIVE_STATUS_nan_MEAN_MEAN,consumer_credit_DAYS_CREDIT_MAX,consumer_credit_DAYS_CREDIT_MIN,consumer_credit_DAYS_CREDIT_ENDDATE_MIN,consumer_credit_DAYS_CREDIT_ENDDATE_MAX,consumer_credit_DAYS_CREDIT_ENDDATE_MEAN,consumer_credit_DAYS_CREDIT_UPDATE_MEAN,consumer_credit_DAYS_CREDIT_UPDATE_MIN,consumer_credit_CREDIT_DAY_OVERDUE_MAX,consumer_credit_CREDIT_DAY_OVERDUE_MEAN,consumer_credit_AMT_CREDIT_MAX_OVERDUE_MEAN,consumer_credit_AMT_CREDIT_MAX_OVERDUE_MAX,consumer_credit_AMT_CREDIT_SUM_MAX,consumer_credit_AMT_CREDIT_SUM_MEAN,consumer_credit_AMT_CREDIT_SUM_SUM,consumer_credit_AMT_CREDIT_SUM_DEBT_MAX,consumer_credit_AMT_CREDIT_SUM_DEBT_MEAN,consumer_credit_AMT_CREDIT_SUM_DEBT_SUM,consumer_credit_AMT_CREDIT_SUM_OVERDUE_MEAN,consumer_credit_AMT_CREDIT_SUM_OVERDUE_MAX,consumer_credit_AMT_CREDIT_SUM_LIMIT_MEAN,consumer_credit_AMT_CREDIT_SUM_LIMIT_SUM,consumer_credit_AMT_ANNUITY_MAX,consumer_credit_AMT_ANNUITY_MEAN,consumer_credit_CNT_CREDIT_PROLONG_SUM,consumer_credit_CNT_CREDIT_PROLONG_ANY,consumer_credit_MONTHS_BALANCE_MIN_MIN,consumer_credit_MONTHS_BALANCE_MAX_MAX,consumer_credit_MONTHS_BALANCE_SIZE_MEAN,consumer_credit_STATUS_0_ANY_ANY,consumer_credit_STATUS_0_ANY_MEAN,consumer_credit_STATUS_0_MEAN_MEAN,consumer_credit_STATUS_1_A

#### CC Balance Var preparation

In [107]:
credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0
